In [12]:
from multiprocessing import Process,Queue
from random import random
from time import sleep
import matplotlib.pyplot as plt
import numpy as np

In [13]:
class Blockchain(Process):
    def __init__(self,bid,queuelist_r,queuelist_s,p1,p2):
        Process.__init__(self)
        self.bid=bid
        self.recvchannel=queuelist_r
        self.sendchannel=queuelist_s
        self.sleeptime=0.1       
        self.networksize=len(queuelist_r)
        self.view=[0 for i in range(self.networksize)]
        self.contract=[dict() for i in range(self.networksize)]
        self.height=0
        self.crossrate=p1
        self.gossiprate=p2
        self.crossseq=0
        self.update=0
        self.gossiptimes=0
        self.sentmessages=0
        
    def state(self):
        print("ID:"+str(self.bid))
        print("Interval:"+str(self.sleeptime))
        print("crossrate:"+str(self.crossrate))
        print("height:"+str(self.height))
        print("_____________________________")
       
    def run(self):
        record=np.zeros((10000,3),dtype=np.int)
        for i in range(10000):
            self.receive()
            self.send()
            self.update=0
            sleep(self.sleeptime)
            record[i]=[self.sentmessages,self.gossiptimes,self.networksize*(i+1)-sum(self.view)]
        title="data/"+str(self.networksize)+"_"+str(self.crossrate)+"_"+str(self.bid)+"_"+str(self.gossiprate)+".npy"
        np.save(title,record)
            
    
    def receive(self):
        for i in range(self.networksize):
            while not self.recvchannel[i].empty():
                Msg=self.recvchannel[i].get()
                if Msg.tag ==0:
                    self.updateview(Msg)
                elif Msg.des==self.bid:
                    self.response(Msg)
                    
    
    def send(self):
        while random()<self.crossrate:
            target=int(self.networksize*random())
            dest=int(random()*self.networksize)
            if dest == self.bid:
                dest=(dest+1)%self.networksize
            Msg=Message(self.bid,dest,1,(self.crossseq, self.height ,self.view[dest]+1)) #target blockchain claimed to commit at next block
            self.contract[self.bid][self.crossseq]=self.view[dest]+1 # add contract to the dcit
            self.crossseq+=1
            self.sendchannel[Msg.des].put(Msg)
            self.sentmessages+=1
        self.height+=1
        self.view[self.bid]=self.height
        if self.update ==1:
            self.gossip(Message(self.bid,0,0,self.view))
    
    def response(self,Msg):
        payload = Msg.info 
        if self.view[Msg.src] < payload[1]:
            self.view[Msg.src]= payload[1]                    
            self.update=1
        if Msg.tag ==1: #receive a request (seq, srch, targeth)
            if self.height < payload[2]: # request failed
                myresponse=Message(self.bid,Msg.src,2,(payload[0], payload[2],False))
            else: #request success
                if payload[0] in self.contract[Msg.src]:
                    del self.contract[Msg.src][payload[0]] #contract finished
                else:
                    self.contract[Msg.src][payload[0]]=payload[1] #add contract (seq,height)
                    self.view[Msg.src]=max(self.view[Msg.src],payload[1])
                myresponse=Message(self.bid,Msg.src,2,(payload[0], payload[2],True))
            
        else: #receive a response (seq,targeth, bool )
            if payload[2]:
                if payload[0] in self.contract[self.bid]: #first round
                    myresponse=Message(self.bid,Msg.src,1,(payload[0],self.height ,self.view[Msg.src]+1))
                    del self.contract[self.bid][payload[0]]
                else:
                    #contract finish
                    myresponse = None
            else: #not reach, request again
                myresponse=Message(self.bid,Msg.src,1,(payload[0], self.height, payload[1]))
                
        if myresponse !=None:
            self.sentmessages+=1
            self.sendchannel[Msg.src].put(myresponse)     
                
    def updateview(self,Msg):
        newview=Msg.info
        for i in range(self.networksize):
            comp=self.view[i]-newview[i]
            if comp<0:
                self.view[i]=newview[i]
                self.update =1
    
    def gossip(self, Msg):
        for i in range(self.networksize):
            if i !=self.bid and random()>self.gossiprate:
                self.sendchannel[i].put(Msg)
                self.gossiptimes+=1
        
                

        
            
        

In [9]:
class Message:
    # tag =0 update message
    # tag =1 request
    # tag =2 response
    def __init__(self,source,dest,tag,content):
        self.src=source
        self.des=dest
        self.tag=tag
        self.info=content
            
    def update(self,newview):
        if self.tag ==0 :
            self.info=newview

    

In [10]:

def MultiBlockchainTest(n,p1,p2):
    Network = [[Queue() for i in range(n)] for j in range(n)]
    Blockchainlist=[Blockchain(i,Network[i],[Channel[i] for Channel in Network],p1,p2) for i in range(n)]
    result=[]
    for BC in Blockchainlist:
        BC.state()
    for BC in Blockchainlist:
        BC.start()
    for BC in Blockchainlist:
        BC.join()
        
    



In [ ]:
MultiBlockchainTest(3,0.1,0.9)
MultiBlockchainTest(5,0.1,0.9)
MultiBlockchainTest(10,0.1,0.9)
MultiBlockchainTest(5,0.2,0.9)
MultiBlockchainTest(5,0.4,0.9)
MultiBlockchainTest(5,0.1,0.8)
MultiBlockchainTest(5,0.1,0.7)

ID:0
Interval:0.1
crossrate:0.1
height:0
_____________________________
ID:1
Interval:0.1
crossrate:0.1
height:0
_____________________________
ID:2
Interval:0.1
crossrate:0.1
height:0
_____________________________


# 